In [1]:
import pandas as pd
import numpy as np
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [6]:
%pwd

'C:\\Users\\pluto\\code_project_portfolio_git\\countries_analysis\\analysis_clean_energy'

# Preparation of data for Power BI report
---

### Renewable energy source production data sets

- [source](http://data.un.org/Explorer.aspx) for renewable energy data

In [2]:
thermal = pd.read_csv('../data/thermal_production_total.csv', parse_dates=['Year'])
wind = pd.read_csv('../data/wind_production_total.csv', parse_dates=['Year'])
solar = pd.read_csv('../data/solar_electricity_production_total.csv', parse_dates=['Year'])
nuclear = pd.read_csv('../data/nuclear_electricity_production_total.csv', parse_dates=['Year'])
ocean = pd.read_csv('../data/ocean_electricity_production_total.csv', parse_dates=['Year'])

### Population data set

The next import is a little different and will require some reshaping to fit a join to all the rest
- [source](https://population.un.org/wpp/Download/Standard/CSV/) used for total population (medium variant .csv)

In [11]:
# import and remove extra columns
pop = pd.read_csv('../data/world_populations/API_SP.POP.TOTL_DS2_en_csv_v2_511378.csv')
pop.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 64', '2017', '2018', '2019'], axis=1,inplace=True)
pop = pop.rename(columns={'Country Name': 'country'}).set_index('country')
pop.columns = [int(year) for year in pop.columns]
pop = pop.loc[:,1990:].head()

# transform the dataframe
pop = (
    pop.unstack()
    .swaplevel()
    .to_frame()
    .sort_index()
    .rename(columns={0: 'population'})
)

# name index and reset index for join to main df
pop.index.rename(['country', 'year'], inplace=True)
# pop.reset_index()

In [9]:
pop.reset_index().to_csv('./total_population_cleaned.csv', index=False)

---

Function to normalize renewable energy source data sets

In [17]:
def normalize_dfs(df):
    df.columns = ['country', 'Commodity', 'year', 'Unit', df.iloc[0, 1].split(' ')[3].strip(',') + '_mil_kwh', 'Footnotes']
    new = df.iloc[:-2, [0, 2, 4]]
    new.year = pd.to_datetime(new.year).dt.year
    new.set_index(['country', 'year'], inplace=True)
    return new

In [18]:
thermal = normalize_dfs(thermal)
wind = normalize_dfs(wind)
solar = normalize_dfs(solar)
nuclear = normalize_dfs(nuclear)
ocean = normalize_dfs(ocean)

In [19]:
print(f"thermal: {thermal.shape}")
print(f"wind: {wind.shape}")
print(f"solar: {solar.shape}")
print(f"nuclear: {nuclear.shape}")
print(f"ocean: {ocean.shape}")

thermal: (24923, 1)
wind: (1609, 1)
solar: (1268, 1)
nuclear: (821, 1)
ocean: (68, 1)


---

## Join 
- `thermal`
- `wind` 
- `solar` 
- `nuclear` 
- `ocean`
- `pop`

In [20]:
joined = thermal.join([wind, solar, nuclear, ocean], how='outer').fillna(0)
joined.drop_duplicates(inplace=True)
joined['total_kwh'] = joined.sum(axis=1)
joined.head()

thermal_mil_kwh  wind_mil_kwh  solar_mil_kwh  \
country     year                                                 
Afghanistan 1990            364.0           0.0            0.0   
            1991            325.0           0.0            0.0   
            1992            225.0           0.0            0.0   
            1993            220.0           0.0            0.0   
            1994            215.0           0.0            0.0   

                  nuclear_mil_kwh  tide_mil_kwh  total_kwh  
country     year                                            
Afghanistan 1990              0.0           0.0      364.0  
            1991              0.0           0.0      325.0  
            1992              0.0           0.0      225.0  
            1993              0.0           0.0      220.0  
            1994              0.0           0.0      215.0

---
## Test proper export of csv

In [21]:
joined.to_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean.csv')
df = pd.read_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean.csv')
df.head()

,country,year,thermal_mil_kwh,wind_mil_kwh,solar_mil_kwh,nuclear_mil_kwh,tide_mil_kwh,total_kwh
0,Afghanistan,1990,364.0,0.0,0.0,0.0,0.0,364.0
1,Afghanistan,1991,325.0,0.0,0.0,0.0,0.0,325.0
2,Afghanistan,1992,225.0,0.0,0.0,0.0,0.0,225.0
3,Afghanistan,1993,220.0,0.0,0.0,0.0,0.0,220.0
4,Afghanistan,1994,215.0,0.0,0.0,0.0,0.0,215.0


---

## Add columns population and kwh_per_capita and test import

In [22]:
joined = joined.join(pop, how='outer')
joined['kwh_per_capita'] = joined['total_kwh'] / joined['population']
joined.fillna(0, inplace=True)

In [24]:
joined.to_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean_pop_kwh_per_capita.csv')
df = pd.read_csv('../data_gdp_and_others/joined_thermal_wind_solar_nuclear_ocean_pop_kwh_per_capita.csv')
df.head()

,country,year,thermal_mil_kwh,wind_mil_kwh,solar_mil_kwh,nuclear_mil_kwh,tide_mil_kwh,total_kwh,population,kwh_per_capita
0,Afghanistan,1990,364.0,0.0,0.0,0.0,0.0,364.0,12412308.0,0.000029
1,Afghanistan,1991,325.0,0.0,0.0,0.0,0.0,325.0,13299017.0,0.000024
2,Afghanistan,1992,225.0,0.0,0.0,0.0,0.0,225.0,14485546.0,0.000016
3,Afghanistan,1993,220.0,0.0,0.0,0.0,0.0,220.0,15816603.0,0.000014
4,Afghanistan,1994,215.0,0.0,0.0,0.0,0.0,215.0,17075727.0,0.000013


In [40]:
df.groupby('country')['total_kwh'].sum().to_frame().nlargest(15, 'total_kwh')

,total_kwh
country,
United States,435225708.0
China,121608117.5
France,119287088.0
Japan,107080349.0
Germany,79161947.0
Russian Federation,77869698.0
"Korea, Republic of",49564114.0
Canada,39740884.0
United Kingdom,38442146.0
